# Save variables to a new netcdf4 file

This notebook shows how to save some calculations you've performed into a new netcdf file. This is really useful if you want to come back it later.

### Import packages and define functions for calculations

In [4]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
from netCDF4 import Dataset

#### Load in some dummy data
This is necessary so that we have something to output. Here we are choosing the interannual variability in sea level pressure 

In [1]:
# FOR THE GEOGRAPHY CLUSTER

'''Select file and variable names'''
#first of all specify some names
reference_period='1851-1900'
ssp_names=["ssp126", "ssp245", "ssp370","ssp585"]
directory='/data/aod/cvdp_cmip6/geog0121/UKESM1-0-LL_ssps.wrt_%s' %reference_period
#Start with SSP370
# determine it's filename
filename='%s/UKESM1_%s_%s.cvdp_data.1850-2100.nc'%(directory,ssp_names[3],reference_period)

In [2]:
#FOR YOUR OWN LAPTOP
'''Select file and variable names'''
filename="Data/UKESM1_ssp370_1851-1900.cvdp_data.1850-2100.nc"
#Download the files if they are not already here
import os
import urllib
if not os.path.isfile(filename):
    !mkdir Data
    urllib.request.urlretrieve("http://www2.geog.ucl.ac.uk/~ucfaccb/geog0121/downloads/UKESM1_ssp370_1851-1900.cvdp_data.1850-2100.nc", filename)
    

A subdirectory or file Data already exists.


In [5]:
# open the file to get the data
ssp370=xr.open_dataset(filename)
field_for_output=ssp370['psl_spatialstddev_ann']
field_for_output

<xarray.DataArray 'psl_spatialstddev_ann' (lat: 144, lon: 192)>
array([[1.657265, 1.657062, 1.656911, ..., 1.657851, 1.657638, 1.657439],
       [1.588344, 1.588433, 1.588682, ..., 1.588519, 1.588354, 1.588306],
       [1.576005, 1.576614, 1.577273, ..., 1.574328, 1.574547, 1.575317],
       ...,
       [2.099676, 2.10872 , 2.117839, ..., 2.073099, 2.081814, 2.090667],
       [2.16448 , 2.170044, 2.175649, ..., 2.14832 , 2.153625, 2.158985],
       [2.229281, 2.231105, 2.232952, ..., 2.223978, 2.225705, 2.227485]],
      dtype=float32)
Coordinates:
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
Attributes:
    standard_name:  air_pressure_at_mean_sea_level
    long_name:      psl standard deviation (annual)
    comment:        Sea Level Pressure
    units:          hPa
    original_name:  mo: (stash: m01s16i222, lbproc: 128)
    history:        2019-04-05T16:01:24Z altered by CMOR: replaced missing va...

This field has a sufficient amount metadata. So we can just output it directly using xarray

In [6]:
field_for_output.to_netcdf('my_xarray_filename.nc','w')

If your calculation had ended up stripping out the metadata, then you need to do things in a more longwinded fashion, but NetCDF4's Dataset command

In [7]:
# Adding some latitudes and longitudes into my workspace
lon=ssp370['lon']
lat=ssp370['lat']

#Create a nc file to save data
d=Dataset('my_netcdf4_filename.nc','w',format='NETCDF4')

#First write out the required dimensions to it
d.createDimension('lon',lon.shape[0])  
d.createDimension('lat',lat.shape[0])  
d.createVariable("lon",'f',("lon"))  
d.createVariable("lat",'f',("lat"))  
d.variables['lat'][:]=lat     
d.variables['lon'][:]=lon

#Then you can write the variable itself
variable_name='psl_spatialstddev_ann'
d.createVariable(variable_name,'f',('lat','lon'))
d.variables[variable_name][:]=field_for_output #variable you want to save    

#Finally close the dataset
d.close()

And now we can look at what netcdf files are in our directory

In [10]:
import os
items = os.listdir(".")

#loop over all the files and only print the ones ending with .nc
for names in items:
    if names.endswith(".nc"):
        print(names)

my_netcdf4_filename.nc
my_xarray_filename.nc
